In [1]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [3]:
number_of_tweets = 200
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


In [4]:
df = pd.DataFrame({"tweets":tweets})
df

,tweets
0,RT @CAPEUSA: DRAMAWORLD season 2 launches THIS...
1,@AllThingTravel @BarcaExplore @BarcelonaInfoES...
2,@toddbarry Do you like #Amazon Gift Card! Grea...
3,Amazon deforestation: Record high destruction ...
4,@TheAcademicWino Do you like #Amazon Gift Card...
...,...
395,Does MacKenzie Scott Still Own Any Amazon (AMZ...
396,(US)In Stock: Asus TUF RX 6500 XT Gaming - 349...
397,(US)In Stock: XFX Speedster QICK210 Radeon RX ...
398,RT @TheVivekSinghal: Amazon stock was at $106 ...


In [6]:
df = df[~df.tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [9]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [10]:
df['clean_tweets'] = df['tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='tweets')
df = df.reset_index(drop=True)
df

,tweets,clean_tweets
0,@AllThingTravel @BarcaExplore @BarcelonaInfoES...,Do like Amazon Gift Card Great News This Amazo...
1,@toddbarry Do you like #Amazon Gift Card! Grea...,Do like Amazon Gift Card Great News This Amazo...
2,Amazon deforestation: Record high destruction ...,Amazon deforestation Record high destruction t...
3,@TheAcademicWino Do you like #Amazon Gift Card...,Do like Amazon Gift Card Great News This Amazo...
4,Corporate titans like Amazon are raising the s...,Corporate titan like Amazon raising salary sta...
...,...,...
221,"@MadamMiaow Ah well, the US has been printing ...",Ah well US printing money like tomorrow gone o...
222,Does MacKenzie Scott Still Own Any Amazon (AMZ...,Does MacKenzie Scott Still Own Any Amazon AMZN...
223,(US)In Stock: Asus TUF RX 6500 XT Gaming - 349...,US In Stock Asus TUF RX XT Gaming ATC Link As ...
224,(US)In Stock: XFX Speedster QICK210 Radeon RX ...,US In Stock XFX Speedster QICK Radeon RX XT AT...


In [13]:
# tokenization
tokenized_doc = df['clean_tweets'].fillna('').apply(lambda x: x.split())

# remove stop-words
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stopwords.words('english')])

# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_tweets'] = detokenized_doc

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_tweets'])

In [24]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=42)
lsa = svd_model.fit_transform(X)

In [25]:
# Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["tweets"] = df['clean_tweets']
display(topic_encoded_df[["tweets", "topic_1", "topic_2"]])

,tweets,topic_1,topic_2
0,Do like Amazon Gift Card Great News This Amazo...,0.0770778764437440,0.1024271858257332
1,Do like Amazon Gift Card Great News This Amazo...,0.0770778764437440,0.1024271858257332
2,Amazon deforestation Record high destruction t...,0.0290793760593020,0.8208647067193138
3,Do like Amazon Gift Card Great News This Amazo...,0.0770778764437440,0.1024271858257332
4,Corporate titan like Amazon raising salary sta...,0.0111064278215990,0.0133382595340869
...,...,...,...
221,Ah well US printing money like tomorrow gone o...,0.0235221504326709,0.0140882874803430
222,Does MacKenzie Scott Still Own Any Amazon AMZN...,0.0429633415380288,0.0186223245523313
223,US In Stock Asus TUF RX XT Gaming ATC Link As ...,0.7661248549908103,-0.0127836494976651
224,US In Stock XFX Speedster QICK Radeon RX XT AT...,0.8086738701278114,-0.0202120579211021


In [28]:
# Features or words used as features
dictionary = vectorizer.get_feature_names_out()
dictionary

array(['ac', 'accelerate', 'accepted', ..., 'zendesk', 'zenfam', 'zoom'],
      dtype=object)

In [29]:
# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns =dictionary).T